In [2]:
import os
os.chdir(os.path.join(os.getcwd(), '..'))
os.getcwd()

'/home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA'

In [18]:
import glob

from src.utils import get_dataset
from src.utils import select_data

import scipy.io as sio
from scipy.signal import resample
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

from keras.utils import to_categorical

from keras.preprocessing import sequence

from keras.models import Sequential
from keras.models import Model

from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import UpSampling1D
from keras.layers import Concatenate

from keras.layers import Conv3D
from keras.layers import MaxPooling3D
from keras.layers import ZeroPadding3D

from keras.optimizers import Adam

from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

In [3]:
DATA_PATH = os.path.join(os.getcwd(), 'data')
os.listdir(DATA_PATH)

['Depth', 'RGB_images', 'Inertial', 'Skeleton', 'RGB', 'Depth_numpy']

In [4]:
depth_path, _,inertial_path, skeleton_path,rgb_path,_ = get_dataset(DATA_PATH)

In [5]:
activities = list(range(1,28))

In [26]:
temp = []
for i in range(len(depth_path)):
    temp.append(sio.loadmat(depth_path[i])['d_depth'].shape[-1])

In [28]:
np.array(temp).min()

41

# Saving them as npy files

In [39]:
def get_action_number(single_path):
    return int(single_path.split('/')[-1].split('_')[0][1:])
def get_subject_number(single_path):
    return int(single_path.split('/')[-1].split('_')[1][1:])
def get_trial_number(single_path):
    return int(single_path.split('/')[-1].split('_')[2][1:])

def get_file_name(single_path):
    '''
    without the .fmt
    '''
    return single_path.split('/')[-1].split('.')[0]

In [29]:
train_or_val = 'train'
resample_len = 41

In [42]:
depth_path[0]

'/home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth/a1_s1_t1_depth.mat'

In [47]:
for depth_single_path in depth_path:
    if get_subject_number(depth_single_path) in [1,3,5,7]:
        train_or_val = 'train'
    else:
        train_or_val = 'val'
    
    class_num = get_action_number(depth_single_path) - 1
    
    SAVE_PATH = os.path.join(os.getcwd(), 'data','Depth_numpy', train_or_val, str(class_num))
    print("Save_path: {}".format(SAVE_PATH))
    if not os.path.exists(SAVE_PATH):
        os.mkdir(SAVE_PATH)
    
    temp = sio.loadmat(depth_single_path)['d_depth']
    temp = temp.transpose([2,0,1])
    start = int((len(temp) - resample_len) /2)
    end = int((len(temp) - resample_len) /2 + resample_len)
    save_npy = temp[start:end]
    
    print("Saving file: {}".format(get_file_name(depth_single_path)))
#     print(os.path.join(SAVE_PATH, get_file_name(depth_single_path) + '.npy'))
    np.save(os.path.join(SAVE_PATH, get_file_name(depth_single_path) + '.npy'),save_npy)

Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/0
Saving file: a1_s1_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/0
Saving file: a1_s1_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/0
Saving file: a1_s1_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/0
Saving file: a1_s1_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/0
Saving file: a1_s2_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/0
Saving file: a1_s2_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/0
Saving file:

Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/1
Saving file: a2_s7_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/1
Saving file: a2_s7_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/1
Saving file: a2_s7_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/1
Saving file: a2_s8_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/1
Saving file: a2_s8_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/1
Saving file: a2_s8_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/1
Saving file: a

Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/3
Saving file: a4_s5_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/3
Saving file: a4_s5_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/3
Saving file: a4_s6_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/3
Saving file: a4_s6_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/3
Saving file: a4_s6_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/3
Saving file: a4_s6_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/3
Saving file: a

Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/5
Saving file: a6_s3_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/5
Saving file: a6_s4_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/5
Saving file: a6_s4_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/5
Saving file: a6_s4_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/5
Saving file: a6_s4_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/5
Saving file: a6_s5_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/5
Saving file: a

Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/7
Saving file: a8_s2_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/7
Saving file: a8_s2_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/7
Saving file: a8_s2_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/7
Saving file: a8_s3_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/7
Saving file: a8_s3_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/7
Saving file: a8_s3_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/7
Saving file:

...

Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/19
Saving file: a20_s4_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/19
Saving file: a20_s4_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/19
Saving file: a20_s5_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/19
Saving file: a20_s5_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/19
Saving file: a20_s5_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/19
Saving file: a20_s5_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/19

Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/21
Saving file: a22_s2_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/21
Saving file: a22_s2_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/21
Saving file: a22_s3_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/21
Saving file: a22_s3_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/21
Saving file: a22_s3_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/21
Saving file: a22_s3_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/21

Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/22
Saving file: a23_s8_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/23
Saving file: a24_s1_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/23
Saving file: a24_s1_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/23
Saving file: a24_s1_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/23
Saving file: a24_s1_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/23
Saving file: a24_s2_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/23

Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/24
Saving file: a25_s6_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/24
Saving file: a25_s7_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/24
Saving file: a25_s7_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/24
Saving file: a25_s7_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/24
Saving file: a25_s7_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/24
Saving file: a25_s8_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/24

Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/26
Saving file: a27_s4_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/26
Saving file: a27_s5_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/26
Saving file: a27_s5_t2_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/26
Saving file: a27_s5_t3_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/train/26
Saving file: a27_s5_t4_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/26
Saving file: a27_s6_t1_depth
Save_path: /home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA/data/Depth_numpy/val/26

In [40]:
get_file_name(depth_path[0])

'a1_s1_t1_depth'

## working with npy files

In [19]:
np.save('test.npy',sio.loadmat(depth_path[10])['d_depth'])

In [6]:
np.load('test.npy').shape

(41, 240, 320)

# Model

In [1]:
import random
class VideoGenerator:

    def __init__(self, train_dir, test_dir, dims, batch_size=2, shuffle=True, file_ext=".npy"):
        self.train_dir = train_dir
        self.test_dir = test_dir
        self.frames, self.width, self.height, self.channels = dims
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.file_ext = file_ext

        self.filenames_train = self.get_filenames(train_dir)
        if self.test_dir:
            self.filenames_test = self.get_filenames(test_dir)

        self.classname_by_id = {i: cls for i, cls in
                                enumerate({os.path.basename(os.path.dirname(file)) for file in self.filenames_train})}
        self.id_by_classname = {cls: i for i, cls in self.classname_by_id.items()}

        self.n_classes = len(self.classname_by_id)
        assert self.n_classes == len(
            self.id_by_classname), "Number of unique classes for training set isn't equal to testing set"

    def get_filenames(self, dir):
        filenames = glob.glob(os.path.join(dir, f"**/*{self.file_ext}"))
        return filenames

    def generate(self, train_or_test, rotation_range=None, heigt_shift_range=None, width_shift_range=None,
                 shear_range=None, zoom_range=None, horizontal_flip=None, vertical_flip=None, brightness_range=None):

        if train_or_test == 'train':
            dir = self.train_dir
        elif train_or_test == 'test':
            dir = self.test_dir
        else:
            raise ValueError

        while True:
            filenames = self.get_filenames(dir)
            if self.shuffle:
                random.shuffle(filenames)

            n_batches = int(len(filenames) / self.batch_size)

            for i in range(n_batches):
                # print(f"Slicing {i*self.batch_size}:{(i+1)*self.batch_size}")
                filenames_batch = filenames[i * self.batch_size:(i + 1) * self.batch_size]
                x, y = self.__generate_data_frome_batch_file_names(filenames_batch)
                yield x, y

    def __generate_data_frome_batch_file_names(self, filenames_batch):
        data = []
        labels = []

        for i, filename in enumerate(filenames_batch):
            npy = np.load(filename)
            try:
                npy = npy[npy.files[0]] # If an npz file we need to get the data out using the filename as a key
            except:
                pass

            if len(npy.shape) == 3:  # Add colour channel to B&W images
                npy = np.expand_dims(npy, axis=-1)

            data.append(npy)
            label = os.path.basename(os.path.dirname(filename))
            labels.append(self.id_by_classname[label])

        x = np.stack(data)
        y = self.list_of_integers_to_2d_onehots(labels)
        return x, y



    def list_of_integers_to_2d_onehots(self, integers):
        array = [[1 if integers[sample] == cls else 0 for cls in range(self.n_classes)] for sample in range(len(integers))]
        return np.array(array)

In [4]:
os.getcwd()

'/home/renjie/Documents/MTech/AY1819Sem1/KE5208_SenseMakingAndInsightsDiscovery/CA'

In [10]:
vid_gen = VideoGenerator(train_dir = os.path.join(os.getcwd(), 'data', 'Depth_numpy', 'train'),
                        test_dir = os.path.join(os.getcwd(), 'data', 'Depth_numpy', 'val'),
                        dims = (41,240,320,1),
                        batch_size = 3,
                        shuffle = True,
                        file_ext=".npy")

In [16]:
train_gen = vid_gen.generate('train')
val_gen = vid_gen.generate('test')

(3, 41, 240, 320, 1)

In [17]:
def c3d():
    """
    Build a 3D convolutional network, aka C3D.
        https://arxiv.org/pdf/1412.0767.pdf
    With thanks:
        https://gist.github.com/albertomontesg/d8b21a179c1e6cca0480ebdf292c34d2
    """
    seq_len = 41
    image_shape = (480,640,1)
    
    model = Sequential(name = '3D_convolutional')
    # 1st layer group
    model.add(Conv3D(64, 3, 3, 3, activation='relu',
                     border_mode='same', name='conv1',
                     subsample=(1, 1, 1),
                     input_shape= (seq_len,) + image_shape))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                           border_mode='valid', name='pool1'))
    # 2nd layer group
    model.add(Conv3D(128, 3, 3, 3, activation='relu',
                     border_mode='same', name='conv2',
                     subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool2'))
    # 3rd layer group
    model.add(Conv3D(256, 3, 3, 3, activation='relu',
                     border_mode='same', name='conv3a',
                     subsample=(1, 1, 1)))
    model.add(Conv3D(256, 3, 3, 3, activation='relu',
                     border_mode='same', name='conv3b',
                     subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool3'))
    # 4th layer group
    model.add(Conv3D(512, 3, 3, 3, activation='relu',
                     border_mode='same', name='conv4a',
                     subsample=(1, 1, 1)))
    model.add(Conv3D(512, 3, 3, 3, activation='relu',
                     border_mode='same', name='conv4b',
                     subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool4'))

    # 5th layer group
    model.add(Conv3D(512, 3, 3, 3, activation='relu',
                     border_mode='same', name='conv5a',
                     subsample=(1, 1, 1)))
    model.add(Conv3D(512, 3, 3, 3, activation='relu',
                     border_mode='same', name='conv5b',
                     subsample=(1, 1, 1)))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool5'))
    model.add(Flatten())

    # FC layers group
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(0.5))
    model.add(Dense(len(activities), activation='softmax'))
    
    print(model.summary())
    return model

In [ ]:
model = 